In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pathlib import Path
from sklearn import metrics
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from datetime import datetime
from collections import OrderedDict

In [3]:
import pickle

In [4]:
PATH = Path("/data2/yinterian/multi-task-romain")

In [5]:
out_path = PATH/"encoded_data"

In [8]:
data = pd.read_csv(PATH/"data-jan-2019/data_modelisation_15min_gap.csv")

In [45]:
data.head()

,key,gender,age,sapsii,sofa,care_unit,amine,sedation,ventilation,hr,spo2,abp_sys,abp_dias,abp_mean,y_mean_MAP,y_mean_HR,patient_id
0,1891-3,1,76,22,1,2,0,0,1,80.0,95.0,99.9,49.4,62.5,59.04,80.0,1891
1,1891-3,1,76,22,1,2,0,0,1,80.0,94.9,98.4,48.7,61.3,59.04,80.0,1891
2,1891-3,1,76,22,1,2,0,0,1,80.0,95.0,95.3,48.5,60.3,59.04,80.0,1891
3,1891-3,1,76,22,1,2,0,0,1,80.0,95.0,97.4,48.9,61.3,59.04,80.0,1891
4,1891-3,1,76,22,1,2,0,0,1,76.7,95.7,99.6,50.2,62.8,59.04,80.0,1891


In [15]:
ids = np.random.permutation(np.unique(data.subject_id.values))
len(ids)

2879

In [16]:
ids2index = {v:k for k,v in enumerate(ids)}

In [19]:
data["patient_id"] = data["subject_id"].apply(lambda x: ids2index[x])

In [24]:
data = data.drop(["subject_id", "time_and_date"], axis=1)

In [31]:
data = data.drop(["observation_mean_MAP", "observation_mean_HR"], axis=1)

In [35]:
data = data[data["window"] == "obs"]

In [37]:
data = data.drop(["periode", "window"], axis=1)

In [27]:
data["key"] = data.apply(lambda x:'%s-%s' % (str(x["patient_id"]),str(x["periode"])),axis=1)

In [44]:
data.iloc[0,]

key            1891-3
gender              1
age                76
sapsii             22
sofa                1
care_unit           2
amine               0
sedation            0
ventilation         1
hr                 80
spo2               95
abp_sys          99.9
abp_dias         49.4
abp_mean         62.5
y_mean_MAP      59.04
y_mean_HR          80
patient_id       1891
Name: 0, dtype: object

In [58]:
data.iloc[5000,]

key            2607-107
gender                0
age                  78
sapsii               31
sofa                  6
care_unit             2
amine                 0
sedation              0
ventilation           0
hr                 80.6
spo2                 99
abp_sys            96.1
abp_dias           48.4
abp_mean           61.6
y_mean_MAP        70.12
y_mean_HR         80.14
patient_id         2607
Name: 8330, dtype: object

In [43]:
data = data.rename(columns={"prediction_mean_MAP": "y_mean_MAP", "prediction_mean_HR": "y_mean_HR"})

In [47]:
patients = data.patient_id.unique()

In [63]:
group = np.random.choice([-1, 0, 1], len(patients), p=[0.3, 0.4, 0.3])
age_diff = pd.DataFrame({"patient_id": patients, "group": group})

In [64]:
age_diff

,patient_id,group
0,1891,-1
1,2410,-1
2,736,0
3,2103,-1
4,1270,-1
...,...,...
2874,465,1
2875,414,0
2876,2785,0
2877,832,0


In [55]:
patient2diff = {p:diff for p,diff in zip(patients, group )}

In [65]:
data_age_diff = data.merge(age_diff, on="patient_id")

In [94]:
data.head()

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,y_mean_MAP,y_mean_HR,group
0,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,99.9,49.4,62.5,59.04,80.0,0
1,1891,1891-3,1,75,22,1,2,0,0,1,80.0,94.9,98.4,48.7,61.3,59.04,80.0,0
2,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,95.3,48.5,60.3,59.04,80.0,0
3,1891,1891-3,1,75,22,1,2,0,0,1,80.0,95.0,97.4,48.9,61.3,59.04,80.0,0
4,1891,1891-3,1,75,22,1,2,0,0,1,76.7,95.7,99.6,50.2,62.8,59.04,80.0,0


In [67]:
data_age_diff["age"] = data_age_diff["age"] + data_age_diff["group"] 

In [69]:
data = data_age_diff.drop(["group"], axis=1)

In [71]:
data = data.rename(columns={"sapsii":"x1","sofa":"x2", "care_unit": "x3", "amine": "x4", "sedation": "x5", "ventilation":"x6"}
                  )

In [74]:
data.columns

Index(['key', 'gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'hr',
       'spo2', 'abp_sys', 'abp_dias', 'abp_mean', 'y_mean_MAP', 'y_mean_HR',
       'patient_id'],
      dtype='object')

In [81]:
cols = data.columns.tolist()
cols = [cols[-1]] + cols[:16]

In [82]:
data = data[cols]

In [85]:
data = data.rename(columns={"hr":"xx1", "spo2":"xx2", "abp_sys":"xx3", "abp_dias":"xx4", "abp_mean":"xx5"})

In [87]:
data.to_csv(out_path/"encoded_data.csv")

In [90]:
np.random.seed(3)
patients_ids = np.sort(data.patient_id.unique())
group = np.random.choice(2, len(patients_ids ), p=[0.8, 0.2])
split = pd.DataFrame({"patient_id": patients_ids, "group": group})
   

#with open(out_path/"split.pickle", 'wb') as f:
#    pickle.dump(split, f)

In [93]:
data = data.merge(split, on="patient_id")

In [95]:
train = data.loc[data["group"] == 0]
test = data.loc[data["group"] == 1]

In [100]:
train = train.drop(["group"], axis=1)
test = test.drop(["group"], axis=1)

In [107]:
train.to_csv(out_path/"train.csv", index=False)

In [102]:
test.to_csv(out_path/"test.csv")

In [5]:
test = pd.read_csv(out_path/"test.csv")

In [6]:
test.head()

,Unnamed: 0,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,y_mean_MAP,y_mean_HR
0,1410,1820,1820-5,0,63,18,4,1,0,0,0,76.0,97.0,103.4,63.7,84.9,85.5,73.52
1,1411,1820,1820-5,0,63,18,4,1,0,0,0,76.3,97.0,105.5,64.4,86.3,85.5,73.52
2,1412,1820,1820-5,0,63,18,4,1,0,0,0,74.6,97.1,102.8,63.4,84.2,85.5,73.52
3,1413,1820,1820-5,0,63,18,4,1,0,0,0,72.9,97.0,103.0,62.3,84.0,85.5,73.52
4,1414,1820,1820-5,0,63,18,4,1,0,0,0,73.9,0.0,104.0,62.8,83.9,85.5,73.52


In [61]:
def my_sample(x):
    if len(x) >= 50:
        xs = x.sample(50) 
    else:
        k = int(1.5*len(x))
        xs = x.sample(k,  replace=True)
    xs["index"] = np.array([i+1 for i in range(len(xs))])
    return xs

In [62]:
keys = test.key.unique()
patients = [key.split("-")[0] for key in keys]
df = pd.DataFrame({"key": keys, "patient_id": patients})

In [63]:
count = df.groupby("patient_id", group_keys=False).size()

In [82]:
count.head()

patient_id
1005     31
1007      5
1025      6
1026    138
1028      3
dtype: int64

In [65]:
df.head()

,key,patient_id
0,1820-5,1820
1,1820-6,1820
2,1820-7,1820
3,1820-8,1820
4,1820-9,1820


In [66]:
df_sample = df.groupby("patient_id", group_keys=False).apply(lambda x: my_sample(x))

In [70]:
df_sample.shape

(13019, 3)

In [75]:
df_sample.head()

,key,patient_id,index,new_key
5326,1005-120,1005,1,1005-1
5321,1005-59,1005,2,1005-2
5332,1005-130,1005,3,1005-3
5319,1005-53,1005,4,1005-4
5336,1005-135,1005,5,1005-5


In [74]:
df_sample["new_key"] = df_sample.apply(lambda x:'%s-%s' % (str(x["patient_id"]),str(x["index"])),axis=1)

In [83]:
def process_periods(df, key, new_key):
    period = test[test["key"]==key].copy()
    period = period.iloc[:,1:]
    period["key"] = new_key
    return period

In [99]:
df_list = []
for i in range(df_sample.shape[0]):
    row = df_sample.iloc[i,]
    key = row["key"]
    new_key = row["new_key"]
    period = process_periods(test, key, new_key)
    print(key, new_key, period.shape)
    df_list.append(period)

1005-120 1005-1 (30, 17)
1005-59 1005-2 (30, 17)
1005-130 1005-3 (30, 17)
1005-53 1005-4 (30, 17)
1005-135 1005-5 (30, 17)
1005-116 1005-6 (30, 17)
1005-132 1005-7 (30, 17)
1005-65 1005-8 (30, 17)
1005-35 1005-9 (30, 17)
1005-137 1005-10 (30, 17)
1005-30 1005-11 (30, 17)
1005-54 1005-12 (30, 17)
1005-139 1005-13 (30, 17)
1005-40 1005-14 (30, 17)
1005-40 1005-15 (30, 17)
1005-120 1005-16 (30, 17)
1005-119 1005-17 (30, 17)
1005-41 1005-18 (30, 17)
1005-130 1005-19 (30, 17)
1005-116 1005-20 (30, 17)
1005-141 1005-21 (30, 17)
1005-119 1005-22 (30, 17)
1005-65 1005-23 (30, 17)
1005-122 1005-24 (30, 17)
1005-128 1005-25 (30, 17)
1005-53 1005-26 (30, 17)
1005-33 1005-27 (30, 17)
1005-30 1005-28 (30, 17)
1005-141 1005-29 (30, 17)
1005-135 1005-30 (30, 17)
1005-32 1005-31 (30, 17)
1005-129 1005-32 (30, 17)
1005-129 1005-33 (30, 17)
1005-28 1005-34 (30, 17)
1005-128 1005-35 (30, 17)
1005-59 1005-36 (30, 17)
1005-122 1005-37 (30, 17)
1005-35 1005-38 (30, 17)
1005-123 1005-39 (30, 17)
1005-133 100

1103-58 1103-20 (30, 17)
1103-62 1103-21 (30, 17)
1103-152 1103-22 (30, 17)
1103-56 1103-23 (30, 17)
1103-37 1103-24 (30, 17)
1103-125 1103-25 (30, 17)
1103-95 1103-26 (30, 17)
1103-56 1103-27 (30, 17)
1103-62 1103-28 (30, 17)
1103-129 1103-29 (30, 17)
1103-62 1103-30 (30, 17)
1103-150 1103-31 (30, 17)
1103-87 1103-32 (30, 17)
1103-33 1103-33 (30, 17)
1103-107 1103-34 (30, 17)
1103-58 1103-35 (30, 17)
1103-65 1103-36 (30, 17)
1103-62 1103-37 (30, 17)
1103-152 1103-38 (30, 17)
1103-145 1103-39 (30, 17)
1103-56 1103-40 (30, 17)
1106-11 1106-1 (30, 17)
1108-19 1108-1 (30, 17)
1108-19 1108-2 (30, 17)
1108-17 1108-3 (30, 17)
1108-6 1108-4 (30, 17)
1108-14 1108-5 (30, 17)
1108-24 1108-6 (30, 17)
1108-15 1108-7 (30, 17)
1108-23 1108-8 (30, 17)
1108-19 1108-9 (30, 17)
1108-4 1108-10 (30, 17)
1108-10 1108-11 (30, 17)
1108-20 1108-12 (30, 17)
1108-10 1108-13 (30, 17)
1108-14 1108-14 (30, 17)
1108-4 1108-15 (30, 17)
1108-10 1108-16 (30, 17)
1108-13 1108-17 (30, 17)
1108-10 1108-18 (30, 17)
1108-1

1178-21 1178-23 (30, 17)
1178-22 1178-24 (30, 17)
1178-15 1178-25 (30, 17)
1182-2 1182-1 (30, 17)
1182-22 1182-2 (30, 17)
1182-13 1182-3 (30, 17)
1182-23 1182-4 (30, 17)
1182-13 1182-5 (30, 17)
1182-19 1182-6 (30, 17)
1182-22 1182-7 (30, 17)
1182-21 1182-8 (30, 17)
1182-15 1182-9 (30, 17)
1182-25 1182-10 (30, 17)
1182-24 1182-11 (30, 17)
1182-13 1182-12 (30, 17)
1182-24 1182-13 (30, 17)
1182-22 1182-14 (30, 17)
1182-24 1182-15 (30, 17)
1182-18 1182-16 (30, 17)
1187-22 1187-1 (30, 17)
1187-17 1187-2 (30, 17)
1187-17 1187-3 (30, 17)
1187-17 1187-4 (30, 17)
1187-17 1187-5 (30, 17)
1187-13 1187-6 (30, 17)
1187-24 1187-7 (30, 17)
1187-21 1187-8 (30, 17)
1187-17 1187-9 (30, 17)
1188-33 1188-1 (30, 17)
1188-13 1188-2 (30, 17)
1188-24 1188-3 (30, 17)
1188-19 1188-4 (30, 17)
1188-25 1188-5 (30, 17)
1188-27 1188-6 (30, 17)
1188-9 1188-7 (30, 17)
1188-21 1188-8 (30, 17)
1188-40 1188-9 (30, 17)
1188-33 1188-10 (30, 17)
1188-37 1188-11 (30, 17)
1188-21 1188-12 (30, 17)
1188-22 1188-13 (30, 17)
1188

1239-6 1239-2 (30, 17)
1239-28 1239-3 (30, 17)
1239-31 1239-4 (30, 17)
1239-9 1239-5 (30, 17)
1239-28 1239-6 (30, 17)
1239-25 1239-7 (30, 17)
1239-9 1239-8 (30, 17)
1239-26 1239-9 (30, 17)
1239-32 1239-10 (30, 17)
1239-14 1239-11 (30, 17)
1239-14 1239-12 (30, 17)
1239-2 1239-13 (30, 17)
1239-33 1239-14 (30, 17)
1239-32 1239-15 (30, 17)
1239-18 1239-16 (30, 17)
1239-3 1239-17 (30, 17)
1239-8 1239-18 (30, 17)
1239-21 1239-19 (30, 17)
1239-3 1239-20 (30, 17)
1239-14 1239-21 (30, 17)
1239-30 1239-22 (30, 17)
1239-9 1239-23 (30, 17)
1239-6 1239-24 (30, 17)
1239-28 1239-25 (30, 17)
1239-17 1239-26 (30, 17)
1239-9 1239-27 (30, 17)
1239-15 1239-28 (30, 17)
1239-6 1239-29 (30, 17)
1239-26 1239-30 (30, 17)
1239-14 1239-31 (30, 17)
1239-26 1239-32 (30, 17)
1239-12 1239-33 (30, 17)
1239-22 1239-34 (30, 17)
1239-17 1239-35 (30, 17)
1239-22 1239-36 (30, 17)
1239-23 1239-37 (30, 17)
1239-2 1239-38 (30, 17)
1239-13 1239-39 (30, 17)
1245-28 1245-1 (30, 17)
1245-38 1245-2 (30, 17)
1245-49 1245-3 (30, 17

1295-6 1295-25 (30, 17)
1299-28 1299-1 (30, 17)
1299-52 1299-2 (30, 17)
1299-38 1299-3 (30, 17)
1299-60 1299-4 (30, 17)
1299-54 1299-5 (30, 17)
1299-5 1299-6 (30, 17)
1299-50 1299-7 (30, 17)
1299-54 1299-8 (30, 17)
1299-11 1299-9 (30, 17)
1299-22 1299-10 (30, 17)
1299-52 1299-11 (30, 17)
1299-8 1299-12 (30, 17)
1299-7 1299-13 (30, 17)
1299-33 1299-14 (30, 17)
1299-12 1299-15 (30, 17)
1299-29 1299-16 (30, 17)
1299-29 1299-17 (30, 17)
1299-10 1299-18 (30, 17)
1299-59 1299-19 (30, 17)
1299-33 1299-20 (30, 17)
1299-19 1299-21 (30, 17)
1299-32 1299-22 (30, 17)
1299-6 1299-23 (30, 17)
1299-11 1299-24 (30, 17)
1299-20 1299-25 (30, 17)
1299-55 1299-26 (30, 17)
1299-22 1299-27 (30, 17)
1299-59 1299-28 (30, 17)
1299-63 1299-29 (30, 17)
1299-6 1299-30 (30, 17)
1299-49 1299-31 (30, 17)
1299-24 1299-32 (30, 17)
1299-16 1299-33 (30, 17)
1299-33 1299-34 (30, 17)
1299-20 1299-35 (30, 17)
1299-45 1299-36 (30, 17)
1299-12 1299-37 (30, 17)
1299-8 1299-38 (30, 17)
1299-58 1299-39 (30, 17)
1299-12 1299-40 

...

715-34 715-3 (30, 17)
715-46 715-4 (30, 17)
715-15 715-5 (30, 17)
715-45 715-6 (30, 17)
715-106 715-7 (30, 17)
715-24 715-8 (30, 17)
715-109 715-9 (30, 17)
715-99 715-10 (30, 17)
715-17 715-11 (30, 17)
715-115 715-12 (30, 17)
715-132 715-13 (30, 17)
715-118 715-14 (30, 17)
715-42 715-15 (30, 17)
715-112 715-16 (30, 17)
715-119 715-17 (30, 17)
715-23 715-18 (30, 17)
715-47 715-19 (30, 17)
715-35 715-20 (30, 17)
715-31 715-21 (30, 17)
715-111 715-22 (30, 17)
715-40 715-23 (30, 17)
715-55 715-24 (30, 17)
715-95 715-25 (30, 17)
715-21 715-26 (30, 17)
715-104 715-27 (30, 17)
715-22 715-28 (30, 17)
715-108 715-29 (30, 17)
715-37 715-30 (30, 17)
715-33 715-31 (30, 17)
715-100 715-32 (30, 17)
715-28 715-33 (30, 17)
715-127 715-34 (30, 17)
715-105 715-35 (30, 17)
715-26 715-36 (30, 17)
715-13 715-37 (30, 17)
715-94 715-38 (30, 17)
715-8 715-39 (30, 17)
715-14 715-40 (30, 17)
715-43 715-41 (30, 17)
715-93 715-42 (30, 17)
715-123 715-43 (30, 17)
715-96 715-44 (30, 17)
715-30 715-45 (30, 17)
715-1

782-248 782-25 (30, 17)
782-166 782-26 (30, 17)
782-22 782-27 (30, 17)
782-77 782-28 (30, 17)
782-22 782-29 (30, 17)
782-248 782-30 (30, 17)
782-135 782-31 (30, 17)
782-213 782-32 (30, 17)
782-206 782-33 (30, 17)
782-178 782-34 (30, 17)
782-137 782-35 (30, 17)
782-103 782-36 (30, 17)
782-28 782-37 (30, 17)
782-13 782-38 (30, 17)
782-232 782-39 (30, 17)
782-63 782-40 (30, 17)
782-208 782-41 (30, 17)
782-178 782-42 (30, 17)
782-29 782-43 (30, 17)
782-225 782-44 (30, 17)
782-29 782-45 (30, 17)
782-29 782-46 (30, 17)
782-13 782-47 (30, 17)
782-177 782-48 (30, 17)
782-127 782-49 (30, 17)
782-201 782-50 (30, 17)
782-77 782-51 (30, 17)
782-177 782-52 (30, 17)
782-219 782-53 (30, 17)
782-171 782-54 (30, 17)
782-44 782-55 (30, 17)
782-13 782-56 (30, 17)
782-208 782-57 (30, 17)
782-111 782-58 (30, 17)
782-45 782-59 (30, 17)
782-53 782-60 (30, 17)
782-208 782-61 (30, 17)
782-248 782-62 (30, 17)
782-177 782-63 (30, 17)
788-5 788-1 (30, 17)
788-6 788-2 (30, 17)
788-5 788-3 (30, 17)
788-2 788-4 (30,

865-38 865-1 (30, 17)
865-39 865-2 (30, 17)
865-51 865-3 (30, 17)
865-8 865-4 (30, 17)
865-53 865-5 (30, 17)
865-8 865-6 (30, 17)
865-2 865-7 (30, 17)
865-53 865-8 (30, 17)
865-33 865-9 (30, 17)
865-24 865-10 (30, 17)
865-33 865-11 (30, 17)
865-50 865-12 (30, 17)
865-36 865-13 (30, 17)
865-46 865-14 (30, 17)
865-23 865-15 (30, 17)
865-16 865-16 (30, 17)
865-12 865-17 (30, 17)
865-38 865-18 (30, 17)
865-39 865-19 (30, 17)
865-12 865-20 (30, 17)
865-21 865-21 (30, 17)
865-39 865-22 (30, 17)
865-25 865-23 (30, 17)
865-46 865-24 (30, 17)
865-9 865-25 (30, 17)
865-19 865-26 (30, 17)
865-24 865-27 (30, 17)
865-47 865-28 (30, 17)
865-46 865-29 (30, 17)
865-39 865-30 (30, 17)
865-21 865-31 (30, 17)
865-39 865-32 (30, 17)
865-9 865-33 (30, 17)
868-6 868-1 (30, 17)
868-17 868-2 (30, 17)
868-2 868-3 (30, 17)
868-5 868-4 (30, 17)
868-6 868-5 (30, 17)
868-17 868-6 (30, 17)
868-2 868-7 (30, 17)
868-19 868-8 (30, 17)
868-23 868-9 (30, 17)
88-155 88-1 (30, 17)
88-73 88-2 (30, 17)
88-112 88-3 (30, 17)


905-41 905-45 (30, 17)
905-39 905-46 (30, 17)
905-29 905-47 (30, 17)
905-41 905-48 (30, 17)
905-34 905-49 (30, 17)
906-33 906-1 (30, 17)
906-56 906-2 (30, 17)
906-51 906-3 (30, 17)
906-33 906-4 (30, 17)
906-17 906-5 (30, 17)
906-24 906-6 (30, 17)
906-45 906-7 (30, 17)
906-16 906-8 (30, 17)
906-53 906-9 (30, 17)
906-5 906-10 (30, 17)
906-45 906-11 (30, 17)
906-24 906-12 (30, 17)
906-30 906-13 (30, 17)
906-5 906-14 (30, 17)
906-7 906-15 (30, 17)
906-14 906-16 (30, 17)
906-37 906-17 (30, 17)
906-43 906-18 (30, 17)
906-16 906-19 (30, 17)
906-46 906-20 (30, 17)
906-45 906-21 (30, 17)
906-49 906-22 (30, 17)
906-7 906-23 (30, 17)
906-8 906-24 (30, 17)
906-14 906-25 (30, 17)
906-36 906-26 (30, 17)
906-14 906-27 (30, 17)
906-14 906-28 (30, 17)
906-8 906-29 (30, 17)
906-7 906-30 (30, 17)
906-35 906-31 (30, 17)
906-43 906-32 (30, 17)
906-17 906-33 (30, 17)
906-8 906-34 (30, 17)
906-33 906-35 (30, 17)
906-46 906-36 (30, 17)
906-15 906-37 (30, 17)
906-30 906-38 (30, 17)
906-35 906-39 (30, 17)
906-2

977-19 977-38 (30, 17)
977-36 977-39 (30, 17)
977-95 977-40 (30, 17)
977-9 977-41 (30, 17)
977-34 977-42 (30, 17)
977-38 977-43 (30, 17)
977-15 977-44 (30, 17)
977-89 977-45 (30, 17)
977-77 977-46 (30, 17)
977-63 977-47 (30, 17)
977-85 977-48 (30, 17)
977-98 977-49 (30, 17)
977-72 977-50 (30, 17)
980-259 980-1 (30, 17)
980-189 980-2 (30, 17)
980-240 980-3 (30, 17)
980-226 980-4 (30, 17)
980-162 980-5 (30, 17)
980-137 980-6 (30, 17)
980-82 980-7 (30, 17)
980-32 980-8 (30, 17)
980-113 980-9 (30, 17)
980-58 980-10 (30, 17)
980-157 980-11 (30, 17)
980-38 980-12 (30, 17)
980-20 980-13 (30, 17)
980-256 980-14 (30, 17)
980-31 980-15 (30, 17)
980-247 980-16 (30, 17)
980-53 980-17 (30, 17)
980-90 980-18 (30, 17)
980-75 980-19 (30, 17)
980-236 980-20 (30, 17)
980-255 980-21 (30, 17)
980-34 980-22 (30, 17)
980-99 980-23 (30, 17)
980-111 980-24 (30, 17)
980-76 980-25 (30, 17)
980-242 980-26 (30, 17)
980-103 980-27 (30, 17)
980-91 980-28 (30, 17)
980-151 980-29 (30, 17)
980-122 980-30 (30, 17)
980-

In [100]:
obs_df = pd.concat(df_list)

In [103]:
obs_df.shape

(390570, 17)

In [104]:
390570/30

13019.0

In [105]:
test_2_students = obs_df.copy()

In [106]:
test_2_students = test_2_students.drop(["y_mean_MAP", "y_mean_HR"], axis=1)
test_2_students.head()

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5
159780,1005,1005-1,0,80,41,4,2,1,0,0,59.0,99.0,118.0,38.0,66.0
159781,1005,1005-1,0,80,41,4,2,1,0,0,60.0,98.0,124.0,41.0,71.0
159782,1005,1005-1,0,80,41,4,2,1,0,0,60.0,97.0,125.0,42.0,70.0
159783,1005,1005-1,0,80,41,4,2,1,0,0,58.0,99.0,123.0,41.0,69.0
159784,1005,1005-1,0,80,41,4,2,1,0,0,60.0,100.0,133.0,47.0,78.0


In [107]:
test_2_students.to_csv(out_path/"data_for_test.csv", index=False)

In [108]:
outcomes = obs_df.loc[:,["key", "y_mean_MAP", "y_mean_HR"]].copy()

In [110]:
outcomes.shape, test_2_students.shape

((390570, 3), (390570, 15))

In [111]:
tt = outcomes.groupby(['key']).mean()

In [113]:
tt.head()

,y_mean_MAP,y_mean_HR
key,,
1005-1,64.793333,53.930000
1005-10,72.346667,73.563333
1005-11,64.423333,65.380000
1005-12,69.230000,60.653333
1005-13,78.553333,74.926667


In [114]:
tt.to_csv(out_path/"test_outcomes.csv")

In [115]:
tt = pd.read_csv(out_path/"test_outcomes.csv")
tt.head()

,key,y_mean_MAP,y_mean_HR
0,1005-1,64.793333,53.930000
1,1005-10,72.346667,73.563333
2,1005-11,64.423333,65.380000
3,1005-12,69.230000,60.653333
4,1005-13,78.553333,74.926667


In [116]:
sample_prediction = tt.copy()

In [117]:
sample_prediction["y_mean_MAP"] = 75.5

In [118]:
sample_prediction["y_mean_HR"] = 70.5

In [119]:
sample_prediction.head()

,key,y_mean_MAP,y_mean_HR
0,1005-1,75.5,70.5
1,1005-10,75.5,70.5
2,1005-11,75.5,70.5
3,1005-12,75.5,70.5
4,1005-13,75.5,70.5


In [122]:
sample_prediction.to_csv(out_path/"sample_submission.csv", index=False)

In [6]:
sample_prediction = pd.read_csv(out_path/"sample_submission.csv")
tt = pd.read_csv(out_path/"test_outcomes.csv")

In [7]:
# testing my score
from sklearn import metrics
y_hat1 = sample_prediction["y_mean_MAP"]
ys1 = tt["y_mean_MAP"]
r2_1 = metrics.r2_score(ys1, y_hat1)


In [8]:
y_hat2 = sample_prediction["y_mean_HR"]
ys2 = tt["y_mean_HR"]
r2_2 = metrics.r2_score(ys2, y_hat2)

In [9]:
(r2_1 + r2_2)/2

-0.4029394131565541

In [10]:
-0.38202*0.7 -0.42149*0.3

-0.39386099999999996